# CityBikes

Send a request to CityBikes for the city of your choice. 

In [2]:
import requests
import pandas as pd

# Define the endpoint URL
url = "http://api.citybik.es/v2/networks"

# Optional: Define any parameters your request might need
params = {}

# Make the request
response_all_api = requests.get(url, {})

# Create dataframe to search for city
networks_df = pd.DataFrame(response_all_api.json().get('networks',{}))

# Define a search function (found online at https://saturncloud.io/blog/how-to-search-for-string-in-all-pandas-dataframe-columns-and-filter/)
def search_string(s, search):
    return search in str(s).lower()

# Search for the string 'al' in all columns (found online at https://saturncloud.io/blog/how-to-search-for-string-in-all-pandas-dataframe-columns-and-filter/)
mask = networks_df.applymap(lambda x: search_string(x, 'toronto'))

# Filter the DataFrame based on the mask (found online at https://saturncloud.io/blog/how-to-search-for-string-in-all-pandas-dataframe-columns-and-filter/)
filtered_df = networks_df.loc[mask.any(axis=1)]

# Define the Citybikes API endpoint for Toronto
toronto_url = "http://api.citybik.es"+networks_df.iloc[filtered_df.index.values[0]]['href']

# Send a GET request to get bike station data for Toronto
response_toronto_api = requests.get(toronto_url)

if response_toronto_api.status_code == 200:
    # city_data = response_toronto_api.json().get("network", {})
    stations = response_toronto_api.json().get("network", {}).get("stations", [])
    
    # if stations:
    #     print("Bike Stations in Toronto:\n")
    #     for station in stations:
    #                 print(f"Station Name: {station['name']}")
    #                 print(f"Available Bikes: {station['free_bikes']}")
    #                 print(f"Empty Docks: {station['empty_slots']}")
    #                 print("\n")
    # else:
    #     print("No bike stations found for Toronto.")
else:
    print("Failed to retrieve data from the Citybikes API.")

Put your parsed results into a DataFrame.

In [3]:
# Create Pandas df of all stations in Toronto
stations_toronto_df = pd.DataFrame(stations)

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
# Filter the dataframe to only                                 
stations_toronto_df_filtered = stations_toronto_df[['free_bikes','longitude', 'latitude']]

# Save a csv
stations_toronto_df_filtered.to_csv(r'C:\Users\akans\Documents\Kierans_Documents\LHL\Statistical-Modelling-Project\data\bike_stations_toronto.csv', index=False)